In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import pathlib
from pathlib import Path
from skimage.measure import label, regionprops, regionprops_table
from rtseg.dotdetection.detect import compute_wavelet_planes
from skimage.io import imread
import torch.nn.functional as F
import matplotlib as mpl
from rtseg.dotdetection.detect import compute_spot_binary_mask
mpl.use('QtAgg')
%matplotlib qt5

### Acquire using pycromanager

In [3]:
from pycromanager import Core, Acquisition

#### Acquire phase image

In [4]:
core = Core()

exposure = core.get_exposure()
core.snap_image()
tagged_image = core.get_tagged_image()

phase_img = np.reshape(tagged_image.pix,
                        newshape=[tagged_image.tags['Height'], tagged_image.tags['Width']])

In [5]:
plt.figure()
plt.imshow(phase_img, cmap='gray')
plt.title(f"{phase_img.shape}")
plt.show()

#### Acquire fluor image after switching in micromanager

In [6]:
core = Core()

exposure = core.get_exposure()
core.snap_image()
tagged_image = core.get_tagged_image()

fluor_img = np.reshape(tagged_image.pix,
                        newshape=[tagged_image.tags['Height'], tagged_image.tags['Width']])

In [7]:
plt.figure()
plt.imshow(fluor_img, cmap='gray')
plt.title(f"{fluor_img.shape}")
plt.show()

#### Do segmentation

In [1]:
from rtseg.cellseg.dataloaders import PhaseContrast
from rtseg.cellseg.utils.transforms import transforms
from rtseg.cellseg.networks import model_dict
from torch.utils.data import DataLoader
from rtseg.cellseg.utils.tiling import get_tiler
from rtseg.cellseg.numerics.vf_to_masks import construct_mask, construct_masks_batch

In [2]:
device='cuda:0'

In [3]:
model_path = Path('C:\\Users\\elflab\\Documents\\Praneeth\\rtseg\\models\\cellseg\\checkpoints\\2024-07-18_15-16-53\\model_val.pt')

NameError: name 'Path' is not defined

In [ ]:
model = model_dict['ResUnet']
model = model.parse(channels_by_scale=[1, 32, 64, 128, 256], num_outputs=[1, 2, 1],
                    upsample_type='transpose_conv', feature_fusion_type='concat',
                    skip_channel_seg=True)
model.load_state_dict(torch.load(model_path))

tiler = get_tiler("dynamic_overlap")
wrapped_model = tiler(model, device=device)
wrapped_model = wrapped_model.to(device)

def run_segnet(image):
    with torch.inference_mode():
        pred_semantic, pred_vf = wrapped_model(image)
    return pred_semantic, pred_vf

# plot the outputs of one image
def plot_inference(pred_semantic, pred_vf, device='cpu'):
    fig, ax = plt.subplots(nrows=1, ncols=3)
    ax[0].imshow(pred_semantic[0][0].cpu().numpy())
    ax[0].set_title("Semantic")
    ax[1].imshow(pred_vf[0][0].cpu().numpy())
    ax[1].set_title("vf_x")
    ax[2].imshow(pred_vf[0][1].cpu().numpy())
    ax[2].set_title("vf_y")
    plt.show()

def tensorize_image(image, device='cuda:0'):
    image_tensor = torch.from_numpy(image).float() / 65535.0
    image_tensor = image_tensor[None, None, :].to(device) # add dimension to play well with (N, C, H, W)
    return image_tensor
    
def segment(image, device='cuda:0'):
    pred_semantic, pred_vf = run_segnet(tensorize_image(image, device=device))
    segmentation_mask = construct_masks_batch(pred_vf, pred_semantic, device=device, store_solutions=False, fast=True)
    return segmentation_mask[0][0].cpu().numpy()

In [37]:
plt.figure()
plt.imshow(phase_img, cmap='gray')
plt.title(f"{phase_img.shape}")
plt.show()

In [35]:
phase_img = np.flipud(phase_img).copy()

In [38]:
phase_img = phase_img.copy()

In [43]:
mask = segment(phase_img)

In [44]:
mask.shape

(736, 2324)

In [45]:
plt.figure()
plt.imshow(mask)
plt.show()

#### Do dot detection

#### Do regionprops and internal coordinates

In [42]:
image = pixels.astype('float32')
image_tensor = torch.from_numpy(image).to(device)
# some transformation
anscombe_image = 2 * torch.sqrt(image_tensor + 3.0/8.0)

In [43]:
w = compute_wavelet_planes(anscombe_image, device=device)

In [44]:
def plot_wavelets(wavelet_planes):
    fig, ax = plt.subplots(nrows=2, ncols=2)
    for i in range(4):
        ax[i//2, i % 2].imshow(wavelet_planes[i].numpy(), cmap='gray')
        ax[i//2, i % 2].set_title(f"i = {i}th plane")
    plt.show()

In [45]:
plot_wavelets(w.cpu())

In [47]:
mask = compute_spot_binary_mask(image,
                                threshold=torch.tensor([ 5.2425,  2.6362,  1.3, 14.9576], device='cuda:0'),
                                wavelet_plane_no=2, device='cuda:0')
plt.figure()
plt.imshow(mask)
plt.show()

In [48]:
spot_stats = regionprops(label(mask), image)
min_spot_area = 0.0 # 4 is default
max_axes_ratio = 1.5
spot_filtered = [item for item in spot_stats if item.area > min_spot_area]

spot_axes_ratio = np.array([spot.axis_major_length / spot.axis_minor_length for spot in spot_filtered])
spot_areas = np.array([spot.area for spot in spot_filtered])
# indices in spot filtered based on area
dot_coords = [spot.centroid_weighted for spot in spot_filtered]
dot_coords_np = np.array(dot_coords)
def plot_dots(fluor_image, dots):
    plt.figure()
    plt.imshow(fluor_image, cmap='gray', vmin=417, vmax=1404)
    plt.scatter(dots[:, 1], dots[:, 0], facecolors='none', edgecolors='b')
    plt.show()
plot_dots(image, dot_coords_np)

#### Hard coded constants

In [41]:
waveletPlanes = 2
levels = 3
fluor_image_path = Path('C:\\Users\\elflab\\Documents\\Praneeth\\waveletCode\\data\\img_000000000.tiff')
seg_image_path = Path('C:\\Users\\elflab\\Documents\\Praneeth\\waveletCode\\data\\img_000000000_mask.tiff')
fluor_img = imread(fluor_image_path).astype('float32')
seg_image = imread(seg_image_path)

In [42]:
from rtseg.dotdetection.detect import compute_spot_binary_mask

In [43]:
image_tensor = torch.from_numpy(fluor_img).to(device)
# some transformation
anscombe_image = 2 * torch.sqrt(image_tensor + 3.0/8.0)
w = compute_wavelet_planes(anscombe_image, device=device)
plot_wavelets(w.cpu())

In [44]:
mask = compute_spot_binary_mask(fluor_img, seg_image, wavelet_plane_no=2, device='cuda:0')

tensor([ 5.2425,  2.6362,  5.2767, 14.9576], device='cuda:0')


In [25]:
plt.figure()
plt.imshow(mask)
plt.show()

In [26]:
spot_stats = regionprops(label(mask), fluor_img)
min_spot_area = 4.0 # 4 is default
max_axes_ratio = 1.5

In [27]:
spot_filtered = [item for item in spot_stats if item.area > min_spot_area]

spot_axes_ratio = np.array([spot.axis_major_length / spot.axis_minor_length for spot in spot_filtered])
spot_areas = np.array([spot.area for spot in spot_filtered])
# indices in spot filtered based on area
dot_coords = [spot.centroid_weighted for spot in spot_filtered]



In [28]:
len(dot_coords)

1079

In [29]:
dot_coords_np = np.array(dot_coords)

In [30]:
dot_coords_np.shape

(1079, 2)

In [31]:
dot_coords_np

array([[ 733.03863318, 1617.92499002],
       [ 214.58543031, 1442.99747214],
       [ 217.75081216, 2527.63563058],
       ...,
       [1359.49490529, 3083.00901372],
       [1368.33591223,  922.41221828],
       [1371.37845871, 2544.50097317]])

In [32]:
len(spot_stats), len(spot_filtered)

(1079, 1079)

In [33]:
overlap_indices = np.where(np.logical_and(spot_axes_ratio > max_axes_ratio, spot_areas >= 2 * min_spot_area))[0]

In [34]:
overlap_indices

array([   0,    2,    4,    9,   13,   17,   18,   20,   22,   24,   25,
         26,   27,   28,   31,   32,   34,   35,   42,   43,   50,   51,
         67,   68,   73,   78,   80,   90,   93,   95,   99,  102,  104,
        106,  109,  114,  117,  119,  122,  123,  125,  127,  128,  132,
        133,  137,  138,  139,  141,  145,  148,  149,  150,  152,  157,
        159,  161,  164,  165,  166,  167,  172,  173,  175,  176,  178,
        181,  182,  191,  195,  201,  204,  206,  207,  208,  209,  212,
        213,  220,  222,  224,  227,  231,  233,  234,  235,  242,  246,
        247,  248,  252,  256,  257,  260,  271,  274,  279,  281,  283,
        284,  286,  287,  289,  290,  294,  301,  303,  304,  305,  316,
        319,  321,  322,  326,  331,  332,  333,  336,  337,  338,  343,
        345,  348,  353,  354,  361,  366,  368,  374,  376,  380,  381,
        382,  384,  390,  398,  401,  411,  414,  421,  424,  426,  427,
        433,  439,  443,  447,  448,  451,  453,  4

In [35]:
orientation = np.array([spot.orientation for spot in spot_filtered])


index = overlap_indices[19]
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].imshow(spot_filtered[index].image, cmap='gray')
ax[1].imshow(spot_filtered[index].image_intensity, cmap='gray')
plt.suptitle(spot_axes_ratio[index])
plt.show()

#### Splitting dots if they meet the overlap criteria

In [37]:
x0, y0 = spot_filtered[index].centroid_weighted

In [38]:
x0, y0

(237.2070909360629, 3134.1268357810413)

#### Plotting dot coordinates

In [39]:
def plot_dots(fluor_image, dots):
    plt.figure()
    plt.imshow(fluor_image, cmap='gray')
    plt.plot(dots[:, 1], dots[:, 0], 'ro')
    plt.show()

In [40]:
plot_dots(fluor_img, dot_coords_np)

In [3]:
fluor_img_tensor = torch.from_numpy(fluor_img)
anscombe_trans_tensor = 2 * torch.sqrt(fluor_img_tensor + 3/8)

In [4]:
anscombe_trans_tensor = 2 * torch.sqrt(fluor_img_tensor + 3/8)

In [5]:
anscombe_trans_image = 2 * np.sqrt(fluor_img+3/8)

In [6]:
anscombe_trans_tensor.device

device(type='cpu')

In [7]:
from rtseg.dotdetection.detect import compute_wavelet_planes

In [8]:
w = compute_wavelet_planes(anscombe_trans_tensor, device='cpu')

In [9]:
w.shape

torch.Size([4, 1404, 3200])

In [10]:
#%timeit w = compute_wavelet_planes(anscombe_trans_tensor, device='cpu')

In [11]:
w_gpu = compute_wavelet_planes(anscombe_trans_tensor, device='cuda:0')

In [12]:
#%timeit w_gpu = compute_wavelet_planes(anscombe_trans_tensor, device='cuda:0')

In [13]:
w.device, w_gpu.device

(device(type='cpu'), device(type='cuda', index=0))

In [14]:
torch.max(torch.abs((w - w_gpu.cpu()))) 

tensor(1.9073e-05)

In [15]:
def plot_wavelets(wavelet_planes):
    fig, ax = plt.subplots(nrows=2, ncols=2)
    for i in range(4):
        ax[i//2, i % 2].imshow(wavelet_planes[i].numpy(), cmap='gray')
        ax[i//2, i % 2].set_title(f"i = {i}th plane")
    plt.show()

In [16]:
plot_wavelets(w_gpu.cpu())

In [17]:
plot_wavelets(w)

In [18]:
seg_image = torch.from_numpy(seg_image)

#### Construction binary mask of the spots using the wavelet planes

There four levels and you need to filter all levels and pick the workplane level as
filtered image. After his clean the filetered image using binaray morphological 
operations

In [116]:
def binary_spot_mask(w, seg_mask, wavelet_plane_no=2, device='cpu', noise_threshold=3.0,
                          noise_level_division=0.7):
    w = w.to(device)
    seg_mask = seg_mask.to(device)
    ind = seg_mask > 0

    w_masked = w[:, ind]
    w_mean = torch.mean(w_masked, dim=1)[:, None]
    noise_level = torch.median(torch.abs(w_masked - w_mean), dim=1).values / noise_level_division
    threshold = noise_threshold * noise_level
    
    filtered_image = torch.zeros_like(w[wavelet_plane_no], device=device)

    filtered_image[torch.abs(w[wavelet_plane_no]) >= threshold[wavelet_plane_no]] = 1;

    filtered_image *= w[wavelet_plane_no]

    binary_mask = torch.zeros_like(filtered_image, device=device)
    binary_mask[filtered_image > 0] = 1

    # clean up the binary mask
    
    
    return binary_mask

In [117]:
spot_mask = binary_spot_mask(w, seg_image, wavelet_plane_no=2, device='cuda:0', noise_threshold=3.0)

In [118]:
spot_mask.device

device(type='cuda', index=0)

In [119]:
spot_mask.shape

torch.Size([1404, 3200])

In [100]:
plt.figure()
plt.imshow(spot_mask.cpu())
plt.show()

#### Clean up kernels

We clean using convolutions on the GPU to do the binary morphological equivalents found in matlab

1. clean --> remove isolated pixels

2. hbreak -> remove two horizontal lines connected by one pixel

3. spur -> remove spurious hanging pixels # we skip the spur for now
4. clean -> clean again
5. thicken -> probably a bit more complicated, dilate the spot later

In [120]:
spot_mask.device

device(type='cuda', index=0)

In [121]:
spot_mask.shape

torch.Size([1404, 3200])

In [122]:
def clean_spots(image):
    clean_kernel = torch.ones((1, 1, 3, 3)).to(image.device)
    conved = F.conv2d(image, clean_kernel, padding='same')
    image[conved == 1] = 0
    return image

def clean_hbreak(image):
    kernel1 = torch.tensor([[1.0, 1.0, 1.0], [0.0, 1.0, 0.0], [1.0, 1.0, 1.0]])[None, None, :].to(image.device)
    kernel2 = torch.tensor([[0.0, 0.0, 0.0], [1.0, 0.0, 1.0], [0.0, 0.0, 0.0]])[None, None, :].to(image.device)
    conved1 = F.conv2d(image, kernel1, padding='same')
    conved2 = F.conv2d(image, kernel2, padding='same')
    result = torch.logical_and(conved1 == 7, conved2 == 2).float()

    find_ones = torch.ones((1, 1, 3, 3)).to(image.device)
    conved = F.conv2d(result, find_ones, padding = 'same')
    image[conved == 1] = 0
    return image

def clean_vbreak(image):
    kernel1 = torch.tensor([[1.0, 0.0, 1.0], [1.0, 1.0, 1.0], [1.0, 0.0, 1.0]])[None, None, :].to(image.device)
    kernel2 = torch.tensor([[0.0, 1.0, 0.0], [0.0, 0.0, 0.0], [0.0, 1.0, 0.0]])[None, None, :].to(image.device)
    conved1 = F.conv2d(image, kernel1, padding='same')
    conved2 = F.conv2d(image, kernel2, padding='same')
    result = torch.logical_and(conved1 == 7, conved2 == 2).float()

    find_ones = torch.ones((1, 1, 3, 3)).to(image.device)
    conved = F.conv2d(result, find_ones, padding = 'same')

    image[conved == 1] = 0
    return image

In [123]:
cleaned = clean_spots(spot_mask[None,None,:])

In [105]:
cleaned.shape

torch.Size([1, 1, 1404, 3200])

In [106]:
plt.figure()
plt.imshow(cleaned.cpu().numpy()[0][0], cmap='gray')
plt.show()

In [107]:
cleaned_breaks = clean_hbreak(cleaned)

In [108]:
plt.figure()
plt.imshow(cleaned_breaks.cpu().numpy()[0][0], cmap='gray')
plt.show()

In [109]:
cleaned_breaks = clean_vbreak(cleaned_breaks)

In [110]:
plt.figure()
plt.imshow(cleaned_breaks.cpu().numpy()[0][0], cmap='gray')
plt.show()

In [111]:
from skimage.segmentation import expand_labels
from skimage.measure import label

In [112]:
cleaned_mask = cleaned_breaks[0][0].cpu().numpy()

In [113]:
final_image = expand_labels(label(cleaned_mask))

In [115]:
plt.figure()
plt.imshow(final_image > 0)
plt.show()

In [39]:
kernel1 = torch.tensor([[1.0, 1.0, 1.0], [0.0, 1.0, 0.0], [1.0, 1.0, 1.0]])[None, None, :].to(spot_mask.device)
conved1 = F.conv2d(cleaned, kernel1, padding='same')

In [47]:
plt.figure()
plt.imshow(conved1.cpu().numpy()[0][0], cmap='gray')
plt.show()

In [41]:
kernel2 = torch.tensor([[0.0, 0.0, 0.0], [1.0, 0.0, 1.0], [0.0, 0.0, 0.0]])[None, None, :].to(spot_mask.device)

In [42]:
conved2 = F.conv2d(cleaned, kernel2, padding='same')

In [48]:
plt.figure()
plt.imshow(conved2.cpu().numpy()[0][0], cmap='gray')
plt.show()

In [49]:
result = torch.logical_and(conved1 == 7, conved2 == 2).float()

In [50]:
find_ones = torch.ones((1, 1, 3, 3)).to(spot_mask.device)
conved = F.conv2d(result, find_ones, padding = 'same')


In [51]:
plt.figure()
plt.imshow(conved.cpu().numpy()[0][0], cmap='gray')
plt.show()

In [53]:
conved.shape

torch.Size([1, 1, 1404, 3200])

In [54]:
spot_mask.shape

torch.Size([1404, 3200])

In [62]:
spot_mask[(conved == 1)[0][0]] = 0.0

In [63]:
plt.figure()
plt.imshow(spot_mask.cpu().numpy())
plt.show()

In [76]:
cleaned2 = clean_hbreak(cleaned)
#cleaned3 = clean_vbreak(cleaned2)

In [77]:
plt.figure()
plt.imshow(cleaned2.cpu().numpy()[0][0], cmap='gray')
plt.show()

In [115]:
torch.ones((1, 1, 3, 3))

tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]])

In [ ]:
def gen_image(shape=(100, 100)):
    prob_dist = torch.ones(shape) * 0.5
    dist = torch.distributions.bernoulli.Bernoulli(probs=prob_dist)
    return dist.sample()[None, None, :]

In [80]:
image = gen_image()
plt.figure()
plt.imshow(image[0][0].numpy(), cmap='gray')
plt.show()

In [81]:
def clean_spots(image):
    clean_kernel = torch.ones((1, 1, 3, 3)).to(image.device)
    conved = F.conv2d(image, clean_kernel, padding='same')
    image[conved == 1] = 0
    return image

In [82]:
plt.figure()
plt.imshow(clean_spots(image)[0][0].numpy(), cmap='gray')
plt.show()

In [92]:
plt.figure()
plt.imshow(image[0][0].numpy(), cmap='gray')
plt.show()

#### We do hbreak and vbreak at the same time, why not?

In [94]:
def clean_hbreak(image):
    kernel1 = torch.tensor([[1.0, 1.0, 1.0], [0.0, 1.0, 0.0], [1.0, 1.0, 1.0]])[None, None, :]
    kernel2 = torch.tensor([[0.0, 0.0, 0.0], [1.0, 0.0, 1.0], [0.0, 0.0, 0.0]])[None, None, :]
    conved1 = F.conv2d(image, kernel1, padding='same')
    conved2 = F.conv2d(image, kernel2, padding='same')
    result = torch.logical_and(conved1 == 7, conved2 == 2)
    image[result] = 0
    return image

In [95]:
def clean_vbreak(image):
    kernel1 = torch.tensor([[1.0, 0.0, 1.0], [1.0, 1.0, 1.0], [1.0, 0.0, 1.0]])[None, None, :]
    kernel2 = torch.tensor([[0.0, 1.0, 0.0], [0.0, 0.0, 0.0], [0.0, 1.0, 0.0]])[None, None, :]
    conved1 = F.conv2d(image, kernel1, padding='same')
    conved2 = F.conv2d(image, kernel2, padding='same')
    result = torch.logical_and(conved1 == 7, conved2 == 2)
    image[result] = 0
    return image

In [39]:
plt.figure()
plt.imshow(clean_hbreak(image)[0][0].numpy(), cmap='gray')
plt.show()

NameError: name 'image' is not defined

In [97]:
plt.figure()
plt.imshow(clean_vbreak(image)[0][0].numpy(), cmap='gray')
plt.show()

### Finding spot centroids

In [123]:
from skimage.measure import regionprops, label, regionprops_table

In [124]:
def find_spot_centroids(spot_bin_mask, fluor_image, min_spot_area=0, max_axes_ratio=1.7):
    pass

In [133]:
%timeit spot_stats = regionprops_table(label(bin_mask_cpu), fluor_img, properties=['centroid_weighted', 'axis_major_length', 'axis_minor_length', 'area', 'orientation', 'coords'])

356 ms ± 2.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [164]:
spot_stats = regionprops(label(bin_mask_cpu), fluor_img)

In [165]:
len(spot_stats)

1576

In [173]:
min_spot_area = 5
max_spot_area = 100
max_axes_ratio = 1.7

In [174]:
area = [(i, spot['area']) 
            for i, spot in enumerate(spot_stats) if spot['area'] >=  min_spot_area and spot['area'] <= max_spot_area ]

In [175]:
spots_filtered = [spot_stats[spot_idx] for spot_idx, spot_area in area]

In [176]:
len(spots_filtered)

1420

In [177]:
axes_ratio = [spot['axis_major_length'] / spot['axis_minor_length'] for spot in spots_filtered]

In [183]:
spots_filtered[3]['centroid_weighted']

(218.58425921621495, 489.79624378225094)

In [185]:
overlap_indices = [idx for (idx, axes_ratio) in enumerate(axes_ratio) if axes_ratio > max_axes_ratio]

In [187]:
len(overlap_indices)

357

In [188]:
overlap_indices

[0,
 6,
 7,
 13,
 18,
 21,
 22,
 23,
 25,
 26,
 29,
 30,
 36,
 38,
 42,
 46,
 57,
 67,
 70,
 71,
 76,
 80,
 88,
 91,
 101,
 103,
 110,
 119,
 121,
 129,
 132,
 134,
 137,
 139,
 142,
 144,
 148,
 149,
 150,
 151,
 159,
 164,
 168,
 169,
 178,
 184,
 186,
 189,
 193,
 194,
 198,
 199,
 200,
 205,
 210,
 213,
 215,
 217,
 220,
 222,
 224,
 225,
 226,
 231,
 232,
 236,
 238,
 240,
 241,
 245,
 248,
 249,
 250,
 254,
 255,
 258,
 259,
 266,
 267,
 276,
 277,
 286,
 291,
 294,
 298,
 301,
 304,
 310,
 318,
 319,
 322,
 325,
 326,
 328,
 329,
 331,
 333,
 334,
 353,
 362,
 367,
 369,
 370,
 372,
 374,
 377,
 380,
 385,
 394,
 395,
 400,
 406,
 413,
 417,
 422,
 426,
 431,
 434,
 440,
 441,
 443,
 445,
 448,
 449,
 452,
 460,
 469,
 470,
 472,
 481,
 483,
 499,
 505,
 510,
 527,
 530,
 532,
 541,
 548,
 552,
 554,
 555,
 559,
 564,
 567,
 573,
 576,
 577,
 579,
 582,
 585,
 587,
 590,
 591,
 594,
 596,
 608,
 610,
 618,
 621,
 622,
 625,
 631,
 633,
 639,
 640,
 648,
 653,
 655,
 656,
 659,
 

#### Working with examples

In [253]:
def sd_intensity(regionmask):
    print(f"Region mask shape: {regionmask.shape}")
    return np.mean(regionmask)

In [260]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from skimage.draw import ellipse
from skimage.measure import label, regionprops, regionprops_table
from skimage.transform import rotate


image = np.zeros((600, 600))

rr, cc = ellipse(300, 350, 100, 220)
image[rr, cc] = 1

image = rotate(image, angle=-15, order=0)

rr, cc = ellipse(100, 100, 60, 50)
image[rr, cc] = 1

label_img = label(image)
regions = regionprops(label_img, extra_properties=(sd_intensity,))

In [255]:
regions = regionprops(label_img, extra_properties=(sd_intensity,))

In [257]:
regions[0].sd_intensity

Region mask shape: (119, 99)


0.7973007384772091

In [258]:
regions[1].sd_intensity

Region mask shape: (225, 429)


0.7155969955969956

In [251]:
sd_intensity

<function __main__.sd_intensity(regionmask)>

In [252]:
list(regions[0])

['area',
 'area_bbox',
 'area_convex',
 'area_filled',
 'axis_major_length',
 'axis_minor_length',
 'bbox',
 'centroid',
 'centroid_local',
 'coords',
 'eccentricity',
 'equivalent_diameter_area',
 'euler_number',
 'extent',
 'feret_diameter_max',
 'image',
 'image_convex',
 'image_filled',
 'inertia_tensor',
 'inertia_tensor_eigvals',
 'label',
 'moments',
 'moments_central',
 'moments_hu',
 'moments_normalized',
 'orientation',
 'perimeter',
 'perimeter_crofton',
 'slice',
 'solidity']

In [244]:
plt.figure()
plt.imshow(label_img)
plt.show()

In [245]:
regions[0].label

1

In [246]:
regions[1].label

2

In [ ]:
def sd_intensity(region

In [211]:
label_img

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [212]:
plt.figure()
plt.imshow(label_img)
plt.show()

In [201]:
from skimage.io import imsave

In [208]:
imsave(Path('/home/pk/Documents/waveletCode/data/test.tiff'), label_img.astype('uint16'),
           plugin='tifffile', check_contrast=False)

In [220]:
fig, ax = plt.subplots()
ax.imshow(image, cmap=plt.cm.gray)

for props in regions:
    y0, x0 = props.centroid
    print(f"(y0, x0): ({y0}, {x0})")
    orientation = props.orientation
    print(f"Orientation: {orientation}")
    x1 = x0 + math.cos(orientation) * 0.5 * props.axis_minor_length
    y1 = y0 - math.sin(orientation) * 0.5 * props.axis_minor_length
    x2 = x0 - math.sin(orientation) * 0.5 * props.axis_major_length
    y2 = y0 - math.cos(orientation) * 0.5 * props.axis_major_length

    ax.plot((x0, x1), (y0, y1), '-k', linewidth=2.5)
    ax.plot((x0, x2), (y0, y2), '-r', linewidth=2.5)
    ax.plot(x0, y0, '.g', markersize=15)

    minr, minc, maxr, maxc = props.bbox
    bx = (minc, maxc, maxc, minc, minc)
    by = (minr, minr, maxr, maxr, minr)
    ax.plot(bx, by, '-b', linewidth=2.5)

ax.axis((0, 600, 600, 0))
plt.show()

(y0, x0): (100.0, 100.0)
Orientation: 0.0
(y0, x0): (313.0515975851635, 348.1505219116008)
Orientation: 1.3089215222380706


In [197]:
-1.308 * 180 / math.pi

-74.94287960311168

In [219]:
list(regions[0])

['area',
 'area_bbox',
 'area_convex',
 'area_filled',
 'axis_major_length',
 'axis_minor_length',
 'bbox',
 'centroid',
 'centroid_local',
 'coords',
 'eccentricity',
 'equivalent_diameter_area',
 'euler_number',
 'extent',
 'feret_diameter_max',
 'image',
 'image_convex',
 'image_filled',
 'inertia_tensor',
 'inertia_tensor_eigvals',
 'label',
 'moments',
 'moments_central',
 'moments_hu',
 'moments_normalized',
 'orientation',
 'perimeter',
 'perimeter_crofton',
 'slice',
 'solidity']

In [261]:
import cv2

In [262]:
cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))

array([[0, 0, 1, 0, 0],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [0, 0, 1, 0, 0]], dtype=uint8)